# Import modules

In [29]:
#!pip install tensorflow_hub

import tensorflow as tf


import pandas as pd
import numpy as np
import tensorflow_hub as hub
import logging
import os
import pprint
#import nltk
import keras
import keras.backend as K
import pickle
from sklearn import metrics
from keras.engine import Layer
from sklearn.model_selection import train_test_split
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from keras.layers import Input, Lambda, Dense, Dropout, LSTM
from keras.models import Model, Sequential
#from nltk import word_tokenize

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))



Found GPU at: /device:GPU:0


# Change working directory and CHOOSE DATA SET

In [2]:
os.chdir("D:\Python Notebooks\Term 2\Statistical NLP")

print("training files: project_training_nn.csv, replace nn for [6,12,25,50,100]")

# training data path
path = "project_data\project_training_6.csv"

# validation data path
val_path = "project_data\project_validation.csv"

training files: project_training_nn.csv, replace nn for [6,12,25,50,100]


# Locate data and read data - CHOOSE DATA SET

In [3]:
print("training files: project_training_nn.csv, replace nn for [6,12,25,50,100]")

# Training set
train = pd.read_csv(path, index_col = 0)

# Validation set
val = pd.read_csv(val_path, index_col = 0)

# Testing set
#test = pd.read_csv("\project_data\balanced_test.csv", index_col = 0)


print("# Comments in training data:", len(train))
print("# Comments in validation data:", len(val))
#print("# Comments in testing data:", len(test))

training files: project_training_nn.csv, replace nn for [6,12,25,50,100]
# Comments in training data: 682
# Comments in validation data: 2734


# Data Pre-processing - lowercasing & sentence capping

In [4]:
def preprocess(df, token):
  """
  Inputs: df: a DF with the following columns: ancestor_index, response, label
          token: a boolean which leaves responses either tokenised (1) or untokenised (0)
          
  Output: lower case and sentence cap by 50 words response column 
  """
  # lower case sentence
  
  dataframe = df
  
  if token == 0:
  
    dataframe['response'] = dataframe['response'].str.lower()
  
    # 50 word sentence cap
  
    # split into tokens
    dataframe['response'] = dataframe['response'].str.split()
  
    # Create loop which says if list element > 50 then return first 50 words else
  
    response = df['response'].tolist()
    for i in range(len(response)):
      if len(response[i]) > 50:
        response[i] = (response[i][:50])
      
      else:
        response[i] = (response[i])
  
    dataframe['response'] = response
  
    # Untokenize sentences
    dataframe['response'] = dataframe['response'].str.join(' ')
    
    return dataframe
    
    
  # Case where token = True
  else:
  
    dataframe['response'] = dataframe['response'].str.lower()
  
    # 50 word sentence cap
  
    # split into tokens
    dataframe['response'] = dataframe['response'].str.split()
  
    # Create loop which says if list element > 50 then return first 50 words else
  
    response = df['response'].tolist()
    for i in range(len(response)):
      if len(response[i]) > 50:
        response[i] = (response[i][:50])
      
      else:
        response[i] = (response[i])
  
    dataframe['response'] = response         
      
    return dataframe

# Preprocess datasets

train = preprocess(df = train, token = 0)
val = preprocess(df = val, token = 0)


# Separate features from labels

In [5]:
# Separate comments and labels

X_train = np.asarray(train.loc[:,'response'])

y_train = np.asarray(train.loc[:,'label'])

# Labels are a 2d vector 0 first col and 1 in the next col for categorical cross entropy
y_train = to_categorical(y_train, num_classes=None)

# View the  data
#print("Validation data")
#print(val.head())

# Separate comments and labels

X_val = np.asarray(val.loc[:,'response'])
y_val = np.asarray(val.loc[:,'label'])

# Labels are a 2d vector 0 first col and 1 in the next col for categorical cross entropy
y_val = to_categorical(y_val, num_classes=None)

# Calling Elmo from tfhub

In [6]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=False)
embeddings = elmo(X_train, signature = 'default', as_dict = True)["default"]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0316 13:36:51.215114 15480 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [7]:
# Define elmo embedding, uses the fully pretrained model from tfhub with their own corpus

def ElmoEmbedding(x):
  """Inputs: x - a string / sentence
             
     Output: converts x to a tensor
     
     Choose ["default"] 
     
  """
  return embed(tf.squeeze(tf.cast(x,tf.string)), signature = "default", as_dict = True)["default"]


url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url, trainable = False)

In [8]:
# Elmo embeddings
input_text = Input(shape = (1, ), dtype = "string")
embedding = Lambda(ElmoEmbedding, output_shape = (1024,) )(input_text)

# Tunable parameters

epochs = [1] #[3, 32]

learning_rate = [0.0001]#[0.001, 0.0001]

reg = [0]#[0.01, 0.001]
#dropout = [0.1, 0.2, 0.3, 0.4, 0.5]

# Execute tf session

initializer = keras.initializers.glorot_normal(seed = 0)

# placeholder to store validation accuracies
accs = pd.DataFrame()

with tf.device("/gpu:0"):
  with tf.Session() as sess:
    
    for rep in range(1):
      for ep in epochs:
        for lr in learning_rate:
          for r in reg:
            print("Repeat:", rep,"Epochs:", ep,",", "Learning rate:", lr, ",", "Regulariser:", r)
            
            # Dense layer to pass embeddings through      
            dense = Dense(256, 
                          activation = 'relu', 
                          bias_initializer = initializer, 
                          kernel_initializer = initializer)(embedding)
            # Regularisation to prevent overfitting
            kernel_reg = keras.regularizers.l2(l = r)(embedding) # edit for hyperparam tune   
            # Prediction layer      
            pred = Dense(2, activation = 'softmax')(dense)
            model = Model(inputs = [input_text], outputs = pred)
            # Optimisation method      
            opt = optimizers.adam(lr = lr) # edit for hyperparam tune
            model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])

            K.set_session(sess)

            # Checkpointing
                  
            # Setting the name of the file so that we can load up the model with this file
            filepath = f'weights-best_{lr}_{r}.hdf5' # old name f'weights-best_{lr}_{r}_'
            # old checkpoint name filepath+'{epoch:02d}'+'_'+'{val_acc:02f}'+'.hdf5'
            checkpoint = ModelCheckpoint(filepath, 
                                         monitor = 'val_acc', 
                                         verbose = 1, 
                                         save_best_only = True, 
                                         mode = 'max')

            history = model.fit(X_train, 
                          y_train, 
                          epochs = ep, # edit for hyperparam tune
                          batch_size = 32, # edit for hyperparam tune
                          validation_data = [X_val,y_val], 
                          verbose = 1,
                          callbacks = [checkpoint])

       # Save the historical validation accuracies in df
      accs[f'(rep:{rep}, lr:{lr}, reg:{r})'] = history.history['val_acc']
            
K.clear_session()   

# dump output file 
#with open("accs.pickle", 'wb') as f:
#  pickle.dump(accs, f)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0316 13:36:53.503872 15480 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


Repeat: 0 Epochs: 1 , Learning rate: 0.0001 , Regulariser: 0
Train on 682 samples, validate on 2734 samples
Epoch 1/1
682/682 [==============================] - 33s 48ms/step - loss: 0.7315 - acc: 0.5279 - val_loss: 0.7232 - val_acc: 0.5282

Epoch 00001: val_acc improved from -inf to 0.52816, saving model to weights-best_0.0001_0.hdf5


# Open pickled files from ELMo_TFhub_accs.zip

In [9]:
# Averages 
a_pickle_in1 = open("avg_val_acc_6.pickle","rb")
a_pickle_in2 = open("avg_val_acc_12.pickle","rb")
a_pickle_in3 = open("avg_val_acc_25.pickle","rb")
a_pickle_in4 = open("avg_val_acc_50.pickle","rb")
a_pickle_in5 = open("avg_val_acc_100.pickle","rb")

# Std devs
s_pickle_in1 = open("stdv_acc_6.pickle","rb")
s_pickle_in2 = open("stdv_acc_12.pickle","rb")
s_pickle_in3 = open("stdv_acc_25.pickle","rb")
s_pickle_in4 = open("stdv_acc_50.pickle","rb")
s_pickle_in5 = open("stdv_acc_100.pickle","rb")

# Averages

acc1 = pickle.load(a_pickle_in1)
acc2 = pickle.load(a_pickle_in2)
acc3 = pickle.load(a_pickle_in3)
acc4 = pickle.load(a_pickle_in4)
acc5 = pickle.load(a_pickle_in5)

# Std Devs

std1 = pickle.load(s_pickle_in1)
std2 = pickle.load(s_pickle_in2)
std3 = pickle.load(s_pickle_in3)
std4 = pickle.load(s_pickle_in4)
std5 = pickle.load(s_pickle_in5)

# Identify best configuration for each dataset

In [10]:
print(acc1.idxmax())
print(acc1.max())

# Best model for 6% is with lr = 0.001, reg = 0.01 on epoch 9

lr:0.001,reg:0.01       9
lr:0.001,reg:0.001     20
lr:0.0001,reg:0.01     30
lr:0.0001,reg:0.001    31
dtype: int64
lr:0.001,reg:0.01      0.596050
lr:0.001,reg:0.001     0.593489
lr:0.0001,reg:0.01     0.594879
lr:0.0001,reg:0.001    0.593489
dtype: float64


In [11]:
print(acc2.idxmax())
print(acc2.max())

# Best model for 12% is with lr = 0.0001, reg = 0.01, epoch 25

lr:0.001,reg:0.01      26
lr:0.001,reg:0.001     26
lr:0.0001,reg:0.01     25
lr:0.0001,reg:0.001    15
dtype: int64
lr:0.001,reg:0.01      0.606950
lr:0.001,reg:0.001     0.605779
lr:0.0001,reg:0.01     0.611339
lr:0.0001,reg:0.001    0.610680
dtype: float64


In [12]:
print(acc3.idxmax())
print(acc3.max())

# Best model for 25% is with lr = 0.0001, reg = 0.01, epoch 24

lr:0.001,reg:0.01      28
lr:0.001,reg:0.001     12
lr:0.0001,reg:0.01     24
lr:0.0001,reg:0.001    28
dtype: int64
lr:0.001,reg:0.01      0.616533
lr:0.001,reg:0.001     0.620629
lr:0.0001,reg:0.01     0.626774
lr:0.0001,reg:0.001    0.625969
dtype: float64


In [13]:
print(acc4.idxmax())
print(acc4.max())

# Best model for 50% is with lr = 0.0001, reg = 0.001, epoch 21

lr:0.001,reg:0.01       5
lr:0.001,reg:0.001      5
lr:0.0001,reg:0.01     11
lr:0.0001,reg:0.001    21
dtype: int64
lr:0.001,reg:0.01      0.638698
lr:0.001,reg:0.001     0.638259
lr:0.0001,reg:0.01     0.648866
lr:0.0001,reg:0.001    0.651134
dtype: float64


# Create model architecture and load models

In [21]:
def balanced_accuracy(out, labels):
  
  #'out' should be the logits
  
  paired_pred = []

  for i in np.arange(0, len(out),2):
    if out[i] > out[i+1]:
      paired_pred.append(0)
      paired_pred.append(1)
    else:
      paired_pred.append(1)
      paired_pred.append(0)
  
  return np.sum(np.array(paired_pred) == labels)/len(out)

def ElmoEmbedding(x):
  """Inputs: x - a string / sentence
     Output: converts x to a tensor
     
     Choose ["default"] 
     
  """
  return embed(tf.squeeze(tf.cast(x,tf.string)), signature = "default", as_dict = True)["default"]

url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url, trainable = True)

def create_model(weights_file):
  
  # Obtain Elmo embeddings from tfhub
  input_text = Input(shape = (1, ), dtype = "string")
  embedding = Lambda(ElmoEmbedding, output_shape = (1024,) )(input_text)
  
  # Set initializer for weight and biases
  initializer = keras.initializers.glorot_normal(seed = 0 )
  
  # Set optimizer
  #opt = optimizers.adam(lr = 0.0001) 
  
  # Create dense layer to do prediction after obtaining ELMO embeddings
  dense = Dense(256, 
                activation = 'relu', 
                bias_initializer = initializer, 
                kernel_initializer = initializer)(embedding)
  
  # Regulariser
  
  #kernel_reg = keras.regularizers.l2(l = 0.01)(embedding)
  
  # Output layer
  pred = Dense(2, activation = 'softmax')(dense)
  
  # Construct model
  model = Model(inputs = [input_text], outputs = pred)
  
  # Load weights for model
  wf = weights_file
  model.load_weights(wf)
  
  return model
  

# Unzip ELMo_TFhub_weights.zip

In [22]:
# Load best params

Model6 = create_model("6weights-best_0.001_0.01.hdf5")

Model12 = create_model("12weights-best_0.0001_0.01.hdf5")

Model25 = create_model("25weights-best_0.0001_0.01.hdf5")

Model50 = create_model("50weights-best_0.0001_0.001.hdf5")

Model100 = create_model("100weights-best_0.0001_0.01.hdf5")

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0316 13:40:47.795001 15480 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0316 13:40:49.430770 15480 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0316 13:40:50.471816 15480 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0316 13:40:51.785653 15480 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0316 13:40:53.009681 15480 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


# Testing model on test set

In [26]:
# Locate test sets
test_path = "project_data\\balanced_test.csv"
u_test_path = "project_data\\unbalanced_test.csv"

# Read csv's
test = pd.read_csv(test_path)
u_test = pd.read_csv(u_test_path)

# Process test data

X_test = np.asarray(test.loc[:,'response'])
y_test = np.asarray(test.loc[:,'label'])

y_test = to_categorical(y_test, num_classes=None)

In [27]:
# Make predictions 

preds6 = Model6.predict(x = X_test,
              verbose = 1)

preds12 = Model12.predict(x = X_test,
              verbose = 1)

preds25 = Model25.predict(x = X_test,
              verbose = 1)

preds50 = Model50.predict(x = X_test,
              verbose = 1)

preds100= Model100.predict(x = X_test,
              verbose = 1)

3406/3406 [==============================] - 28s 8ms/step


In [30]:
# Return accuracies

accuracy_6 = metrics.accuracy_score(y_true = np.asarray(test.loc[:,'label']), 
                                  y_pred = np.asarray(np.round(preds6[:,1])))

accuracy_12 = metrics.accuracy_score(y_true = np.asarray(test.loc[:,'label']), 
                                  y_pred = np.asarray(np.round(preds12[:,1])))

accuracy_25 = metrics.accuracy_score(y_true = np.asarray(test.loc[:,'label']), 
                                  y_pred = np.asarray(np.round(preds25[:,1])))

accuracy_50 = metrics.accuracy_score(y_true = np.asarray(test.loc[:,'label']), 
                                  y_pred = np.asarray(np.round(preds50[:,1])))

accuracy_100 = metrics.accuracy_score(y_true = np.asarray(test.loc[:,'label']), 
                                  y_pred = np.asarray(np.round(preds100[:,1])))

In [36]:
print("Testing accuracy")
print("6%", accuracy_6*100, )
print("12%", accuracy_12*100)
print("25%", accuracy_25*100)
print("50%", accuracy_50*100)
print("100%", accuracy_100*100)

a6 = accuracy_6*100
a12 = accuracy_12*100
a25 = accuracy_25*100
a50 = accuracy_50*100
a100 = accuracy_100*100

print("Balanced acc")
print("6%",1-balanced_accuracy(preds6[:,1], y_test[:,1]))
print("12%",1-balanced_accuracy(preds12[:,1], y_test[:,1]))
print("25%",1-balanced_accuracy(preds25[:,1], y_test[:,1]))
print("50%",1-balanced_accuracy(preds50[:,1], y_test[:,1]))
print("100%",1-balanced_accuracy(preds100[:,1], y_test[:,1]))

b6 = (1-balanced_accuracy(preds6[:,1], y_test[:,1]))*100
b12 = (1-balanced_accuracy(preds12[:,1], y_test[:,1]))*100
b25 = (1-balanced_accuracy(preds25[:,1], y_test[:,1]))*100
b50 = (1-balanced_accuracy(preds50[:,1], y_test[:,1]))*100
b100 = (1-balanced_accuracy(preds100[:,1], y_test[:,1]))*100

print("F1 Scores")

print("6%", metrics.f1_score(y_true = np.asarray(test.loc[:,'label']), y_pred = np.asarray(np.round(preds6[:,1]))))
print("12%", metrics.f1_score(y_true = np.asarray(test.loc[:,'label']), y_pred = np.asarray(np.round(preds12[:,1]))))
print("25%", metrics.f1_score(y_true = np.asarray(test.loc[:,'label']), y_pred = np.asarray(np.round(preds25[:,1]))))
print("50%", metrics.f1_score(y_true = np.asarray(test.loc[:,'label']), y_pred = np.asarray(np.round(preds50[:,1]))))
print("100%", metrics.f1_score(y_true = np.asarray(test.loc[:,'label']), y_pred = np.asarray(np.round(preds100[:,1]))))

f6 = metrics.f1_score(y_true = np.asarray(test.loc[:,'label']), y_pred = np.asarray(np.round(preds6[:,1])))
f12 = metrics.f1_score(y_true = np.asarray(test.loc[:,'label']), y_pred = np.asarray(np.round(preds12[:,1])))
f25 = metrics.f1_score(y_true = np.asarray(test.loc[:,'label']), y_pred = np.asarray(np.round(preds25[:,1])))
f50 = metrics.f1_score(y_true = np.asarray(test.loc[:,'label']), y_pred = np.asarray(np.round(preds50[:,1])))
f100 = metrics.f1_score(y_true = np.asarray(test.loc[:,'label']), y_pred = np.asarray(np.round(preds100[:,1])))


Testing accuracy
6% 60.217263652378165
12% 60.011743981209634
25% 63.27069876688197
50% 64.32765707574868
100% 66.88197298884322
Balanced acc
6% 0.6353493834409865
12% 0.6341749853200235
25% 0.6776277157956547
50% 0.7028772753963594
100% 0.7351732237228421
F1 Scores
6% 0.5599220526144852
12% 0.5423387096774194
25% 0.6219401631912965
50% 0.6537475064120832
100% 0.6361290322580645


In [34]:
# Create dataframe with the summarised data and findings

final_df = pd.DataFrame()

final_df['Training size'] = ["6%","12%","25%","50%","100%"]
final_df['Hyper-parameters-(LR,Reg)'] = ["(0.001,0.01)",
                                       "(0.0001,0.01)",
                                       "(0.0001,0.01)",
                                       "(0.0001,0.001)",
                                       "(0.0001,0.01)"]
final_df['F1-Score'] = [f6,f12,f25,f50,f100]
final_df['Test accuracy(%)'] = [a6,a12,a25,a50,a100]
final_df['Balanced Test accuracy(%)'] = [b6,b12,b25,b50,b100]

In [35]:
# See the table
final_df.head()

,Training size,"Hyper-parameters-(LR,Reg)",F1-Score,Test accuracy(%),Balanced Test accuracy(%)
0,6%,"(0.001,0.01)",0.559922,60.217264,63.534938
1,12%,"(0.0001,0.01)",0.542339,60.011744,63.417499
2,25%,"(0.0001,0.01)",0.621940,63.270699,67.762772
3,50%,"(0.0001,0.001)",0.653748,64.327657,70.287728
4,100%,"(0.0001,0.01)",0.636129,66.881973,73.517322


In [ ]:
# Export the dataframe

final_df.to_csv('ELMO_Results.csv')